# Previsão de Deputados Eleitos

- 1 Há desbalanceamento das classes (isto é, uma classe tem muito mais instâncias que outra)? 
    Em que proporção? 
    Quais efeitos colaterais o desbalanceamento de classes pode causar no classificador? 
    Como você poderia tratar isso? (2 pt.)

- 2 Treine: um modelo de regressão logística, uma árvore de decisão e um modelo de adaboost. 
    Tune esses modelos usando validação cruzada e controle overfitting se necessário, 
    considerando as particularidades de cada modelo.  (2 pts.)
    
- 3 Reporte acurácia, precision, recall e f-measure no treino e validação. 
    Há uma grande diferença de desempenho no treino/validação? 
    Como você avalia os resultados? Justifique sua resposta. (2 pt.)
    
- 4 Interprete as saídas dos modelos. 
    Quais atributos parecem ser mais importantes de acordo com cada modelo? (2 pts.)
    
- 5 Envie seus melhores modelos à competição do Kaggle. 
    Faça pelo menos uma submissão. 
    Sugestões para melhorar o modelo: (2 pts.)
        
    - 5.1 Experimente outros modelos (e.g. SVM, RandomForests e GradientBoosting).
    - 5.2 Experimente balancear as classes, caso estejam desbalanceadas.
    - 5.3 Experimente balancear as classes,  caso estejam desbalanceadas

In [2]:
%matplotlib inline

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('seaborn-whitegrid')

In [3]:
def normalizando_dados(dataframe):
    
    numeric_features = dataframe.dtypes[dataframe.dtypes != "object"].index

    skewed_features = dataframe[numeric_features].apply(lambda x: x.dropna().skew())

    skewed_features = skewed_features[skewed_features > 0.75]

    skewed_features = skewed_features.index

    dataframe[skewed_features] = np.log1p(dataframe[skewed_features])

    dataframe = pd.get_dummies(dataframe)

    dataframe = dataframe.fillna(dataframe.mean())
    
    return dataframe

### Importando dados de treino e de teste

In [28]:
train = pd.read_csv('../datasets/train.csv')
test = pd.read_csv('../datasets/test.csv')

## 1 - Há desbalanceamento das classes? 
- Isto é, uma classe tem muito mais instâncias que outra? 


In [53]:
group_situacao = train.groupby('situacao').size().reset_index(name='qtd').sort_values('qtd', ascending=False)
group_situacao

,situacao,qtd
1,nao_eleito,6596
0,eleito,1026


Sim. A quantidade de exemplos na classe dos candidatos **não eleitos** é aproximadamente 6 vezes maior do que a quantidade de exemplos na classe de candidatos **eleitos**.

- **Em que proporção?** 

In [57]:
group_situacao['proporcao'] = group_situacao['qtd'] / sum(group_situacao['qtd'])
group_situacao['porcentagem'] = (group_situacao['qtd'] / sum(group_situacao['qtd']))*100

group_situacao.style.format({
    'porcentagem': '{:.2f}%'
})

,situacao,qtd,proporcao,porcentagem
1,nao_eleito,6596,0.86539,86.54%
0,eleito,1026,0.13461,13.46%


A proporção do desbalanceamento é
- Classe não eleito - 0,87
- Classe eleito - 0,13

- **Quais efeitos colaterais o desbalanceamento de classes pode causar no classificador?**

Overfitting dos dados para a classe com maior quantidade de exemplos, gerando assim um classificador com tendência a classificar os candidatos como **não eleitos**

# Preparando dados para a classificação

In [59]:
y_train = train['situacao']
y_train.head()

0    nao_eleito
1    nao_eleito
2        eleito
3    nao_eleito
4        eleito
Name: situacao, dtype: object

In [60]:
train = train.drop(['media_receita', 'media_despesa', 'situacao'], axis=1)
train = normalizando_dados(train)
train.head()

,ano,sequencial_candidato,quantidade_doacoes,quantidade_doadores,total_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,...,"ocupacao_TÉCNICO DE OBRAS CIVIS, ESTRADAS, SANEAMENTO E ASSEMELHADOS",ocupacao_TÉCNICO EM AGRONOMIA E AGRIMENSURA,ocupacao_TÉCNICO EM EDIFICAÇÕES,ocupacao_TÉCNICO EM INFORMÁTICA,ocupacao_VENDEDOR DE COMÉRCIO VAREJISTA E ATACADISTA,"ocupacao_VENDEDOR PRACISTA, REPRESENTANTE, CAIXEIRO-VIAJANTE E ASSEMELHADOS",ocupacao_VEREADOR,ocupacao_VETERINÁRIO,ocupacao_VIGILANTE,ocupacao_ZOOTECNISTA
0,2006,10001,1.945910,1.945910,9.717218,0.000000,9.105091,8.748464,7.170888,0.0,...,0,0,0,0,0,0,1,0,0,0
1,2006,10002,2.639057,2.639057,10.035699,8.798757,9.615872,6.908755,5.308268,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2006,10002,2.890372,2.833213,11.971121,7.719130,10.438547,11.038461,10.993748,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2006,10002,1.945910,1.945910,8.007074,0.000000,7.048386,7.004991,6.621406,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2006,10003,3.891820,3.891820,11.693754,0.000000,10.837206,0.000000,11.141035,0.0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
test = test.drop(['media_receita', 'media_despesa'], axis=1)
test = normalizando_dados(test)
test.head()

,ano,sequencial_candidato,quantidade_doacoes,quantidade_doadores,total_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,...,"ocupacao_TÉCNICO DE MINERAÇÃO, METALURGIA E GEOLOGIA",ocupacao_TÉCNICO EM AGRONOMIA E AGRIMENSURA,ocupacao_TÉCNICO EM EDIFICAÇÕES,ocupacao_TÉCNICO EM INFORMÁTICA,ocupacao_VENDEDOR DE COMÉRCIO VAREJISTA E ATACADISTA,"ocupacao_VENDEDOR PRACISTA, REPRESENTANTE, CAIXEIRO-VIAJANTE E ASSEMELHADOS",ocupacao_VEREADOR,ocupacao_VETERINÁRIO,ocupacao_VIGILANTE,ocupacao_ZOOTECNISTA
0,2014,10000000135,1.386294,1.386294,7.365813,0.000000,7.313887,0.000000,0.000000,4.394449,...,0,0,0,0,0,0,0,0,0,0
1,2014,10000000142,1.791759,1.791759,8.064951,0.000000,8.039480,0.000000,0.000000,4.394449,...,0,0,0,0,0,0,0,0,1,0
2,2014,10000000158,3.713572,3.663562,12.727227,7.562198,11.092869,11.945818,11.652696,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,2014,10000000161,3.401197,3.401197,11.962218,10.574474,11.138739,9.741027,8.826758,10.126671,...,0,0,0,0,0,0,0,0,0,0
4,2014,10000000163,5.081404,4.990433,13.510444,9.210440,12.757293,12.531776,11.264477,10.126671,...,0,0,0,0,0,0,0,0,0,0


## 2 - Treinando Modelos de Classificaçãlo

#### Funções comuns para validação cruzada e visualização do resíduo

In [63]:
def evaluation_model(model, cv, metric):
    metric_evaluation = np.sqrt(-cross_val_score(model, x_all, y_all, scoring=metric, cv=cv))
    print("{0} is: {1} ".format(scoring, rmse))
    return metric_evaluation

In [64]:
def gerar_vis_validacao(measures, alphas, title_measure):
    matplotlib.rcParams['figure.figsize'] = (8.0, 6.0)
    data = pd.Series(measures, index=alphas)
    data.plot(title = "Validação")
    plt.xlabel("alpha")
    plt.ylabel(title_measure)

### Regressão Logística

### Árvore de Decisão

### Adaboost